<h1>Capstone: Segmenting and Clustering Neighbourhoods in Toronto</h1>

<h4>This notebook will segment and cluster neighborhood data from Toronto, Canada obtained by scraping from a wikipedia page.</h4> 

<h2> Part 1: Building the postal code dataframe</h2>

<h4>Import pandas</h4>

In [1]:
# for converting the parsed data in a pandas dataframe
import pandas as pd

<h4>Parse the tables from the web page into a pandas dataframe and check what is returned.</h4>

In [2]:
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
print(type(df))
len(df)

<class 'list'>


3

<h4>Since there are three tables, df is a list of dataframes. Select the first into a new dataframe and check length.</h4>

In [3]:
df_hoods=df[0]
print(type(df_hoods))
len(df_hoods)

<class 'pandas.core.frame.DataFrame'>


180

<h4>Check first 5 rows.</h4>

In [4]:
df_hoods.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


<h4>Remove rows with Borough "Not assigned" and reset the index. Check the first 5 rows.</h4> 

In [5]:
df_hoods=df_hoods[df_hoods.Borough != 'Not assigned']
df_hoods.reset_index(drop=True, inplace=True)

In [6]:
df_hoods.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


<h4>Check if there are any Neighbourhoods 'Not Assigned'</h4>

In [7]:
df_hoods_NNa = df_hoods[df_hoods.Neighbourhood == 'Not assigned']
len(df_hoods_NNa)

0

<h4>Print the number of rows in the dataframe using the .shape method.</h4>

In [8]:
print(df_hoods.shape[0])

103


<h2>